In [1]:
"""ALLRIGHT METHOD TIME"""

"""
Method to get the data
Requires at least one path/url as string and a name for the to be loaded data

Returns a dict with name of data as key and loaded data as DataFrame as value.
"""
import pandas as pd

def get_data_from_urls(path, name, path_2=None, name_2=None):
    
    data_dict = {}
    
    data = pd.read_csv(path, index_col=0)
    
    data_dict[name] = data
    
    if path_2:
        data_2 = pd.read_csv(path_2, index_col=0)
        
        data_dict[name_2] = data_2
    
    return data_dict

In [5]:
#####
# Test get_data_from_urls
#####
url_data_task_A = "https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_data_all.csv"
url_answers_task_A = "https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_answers_all.csv"

data = get_data_from_urls(url_data_task_A, 'data_a', url_answers_task_A, 'answers_a')




In [12]:
"""
Method to get different words
Uses spacy tokenizer instead of bert
Takes a tuple of string sentences and a parser

Returns a dict with different words as keys, index of word in sentence as value

Helper-Method to get spacy parser
takes a name of language model (?) as string
returns parser
"""
import spacy
import pandas as pd


def get_different_words(sent_tuple, parser):

    sen_0 = parser(sent_tuple[0])
    sen_1 = parser(sent_tuple[1])

    length_0 = len(sen_0)
    length_1 = len(sen_1)

    differ_dict_0 = {}
    differ_dict_1 = {}

    index_of_word = 0

    for token in sen_0:
        word = token.text

        if not word in sen_1.text:
            differ_dict_0[word] = index_of_word
            index_of_word += 1
        else:
            index_of_word += 1

    index_of_word = 0
    for token in sen_1:
        word = token.text

        if not word in sen_0.text:
            differ_dict_1[word] = index_of_word
            index_of_word += 1
        else:
            index_of_word += 1


    differ_dict = (differ_dict_0, differ_dict_1)
    
    return differ_dict

def get_parser(model_name):
    
    try:
        parser = spacy.load(model_name)
        return parser
    except:
        !python3 -m spacy download model_name

        try:
            parser = spacy.load(model_name)
            return parser
        except:
            print('Something went horribly wrong. Check your model')
            




In [11]:
parser = get_parser('en_core_web_lg')


In [17]:
sent_tuple = ('I lol the elephant into the fridge',
                 'I put the turkey into the fridge')
different_words = get_different_words(sent_tuple, parser)

for key, value in different_words[0].items():
    print(type(key))

<class 'str'>
<class 'str'>


In [18]:
"""
Method to get dependents of subj/objs with or without verb
takes a parser, different subj or obj and its index, corresponding sentence, boolean with verb

returns list of dependents
"""

"""BEWARE: DIFFERENT INDIRECT OBJECTS REQUIRE SEPARATE METHOD"""
"""this approach maybe doesnt even make sense...damn, 
maybe we should just throw out the stopwords and then calculate similarities"""

#def get_subj_obj_dependents(parser, different_word, index sentence, with_verb=False):
parser = parser
different_word = 'elephant'
index = different_words[0][different_word]
sentence = sent_tuple[0]
with_verb = False

parse = parser(sentence) 
verb_dependents = []

parent = parse[index].head

if parent.pos_ == 'VERB':
    # now we want all children of the verb with subj and obj labels

    for child in parent.children:
        if child.dep_[-2:] == 'bj':
            verb_dependents.append(child)
        else:
            for nephew in child.children:
                if nephew.dep_[-2:] == 'bj':
                    verb_dependents.append(nephew)
    
    
    

In [19]:
print(verb_dependents)

[I, elephant, fridge]


In [ ]:
"""compute similarity for all right predictions and take average, if novel sentence have similarity below this average
use perplexity rather
"""